In [2]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

2024-02-13 18:12:54.421306: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 18:12:54.599990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 18:12:54.600232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 18:12:54.653646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 18:12:54.829972: I tensorflow/core/platform/cpu_feature_guar

In [22]:
normal = pd.read_csv("final-normal-data-set.csv", nrows=1000)
anormal = pd.read_csv("final-anormal-data-set.csv", nrows=1000)

#inserting column for result 
normal.insert(82, 'result', 0)
anormal.insert(82, 'result', 1)

In [23]:
#combining datasets
frames = [normal, anormal]
df = pd.concat(frames)

In [24]:
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

1425 examples in training, 575 examples for testing.


In [30]:
label = "result"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [31]:
# random forest
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 16 thread(s) for training
Use /tmp/tmpe16g9moa as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'cpu_guest': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'cpu_guest_nice': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'cpu_idle': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'cpu_iowait': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'cpu_irq': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'cpu_nice': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'cpu_softirq': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'cpu_steal': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'cpu_system': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'cpu_total': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'cpu_user': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'diskio_sda1_disk_name': <tf.Tensor 'data_11:0' shape=(None,) dtype=string>, 'diskio_sda1_key': <tf.Tensor 'data_12:0' shape=(None,) d

[INFO 24-02-13 18:31:23.2457 PST kernel.cc:771] Start Yggdrasil model training
[INFO 24-02-13 18:31:23.2458 PST kernel.cc:772] Collect training examples
[INFO 24-02-13 18:31:23.2458 PST kernel.cc:785] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 24-02-13 18:31:23.2461 PST kernel.cc:391] Number of batches: 2
[INFO 24-02-13 18:31:23.2461 PST kernel.cc:392] Number of examples: 1425
[INFO 24-02-13 18:31:23.2496 PST data_spec_inference.cc:305] 1417 item(s) have been pruned (i.e. they are considered out of dictionary) for the column timestamp (0 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 24-02-13 18:31:23.2510 PST kernel.cc:7

Model trained in 0:00:00.332075
Compiling model...
Model compiled.


In [32]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 283ms/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


In [52]:
# nvidia dataset

df_train = pd.read_json("nvidia_train.json")
df_test = pd.read_csv("nvidia_test.csv")
df_train.head()print

,nvidia_smi_log.timestamp,nvidia_smi_log.gpu.pci.tx_util,nvidia_smi_log.gpu.pci.rx_util,nvidia_smi_log.gpu.fb_memory_usage.used,nvidia_smi_log.gpu.fb_memory_usage.free,nvidia_smi_log.gpu.bar1_memory_usage.total,nvidia_smi_log.gpu.bar1_memory_usage.used,nvidia_smi_log.gpu.bar1_memory_usage.free,nvidia_smi_log.gpu.utilization.gpu_util,nvidia_smi_log.gpu.utilization.memory_util,...,nvidia_smi_log.gpu.applications_clocks.graphics_clock,nvidia_smi_log.gpu.applications_clocks.mem_clock,nvidia_smi_log.gpu.default_applications_clocks.graphics_clock,nvidia_smi_log.gpu.default_applications_clocks.mem_clock,nvidia_smi_log.gpu.max_clocks.graphics_clock,nvidia_smi_log.gpu.max_clocks.sm_clock,nvidia_smi_log.gpu.max_clocks.mem_clock,nvidia_smi_log.gpu.max_clocks.video_clock,nvidia_smi_log.gpu.max_customer_boost_clocks.graphics_clock,label
0,Wed Mar 3 15:03:01 2021,0,0,3909,12251,16384,7,16377,100,2,...,1530,877,1312,877,1530,1530,877,1372,1530,0
1,Wed Mar 3 15:04:01 2021,0,2000,3909,12251,16384,7,16377,100,5,...,1530,877,1312,877,1530,1530,877,1372,1530,0
2,Wed Mar 3 15:05:01 2021,1000,2000,3909,12251,16384,7,16377,100,5,...,1530,877,1312,877,1530,1530,877,1372,1530,0
3,Wed Mar 3 15:06:01 2021,0,2000,3909,12251,16384,7,16377,100,6,...,1530,877,1312,877,1530,1530,877,1372,1530,0
4,Wed Mar 3 15:07:01 2021,1000,2000,3909,12251,16384,7,16377,100,6,...,1530,877,1312,877,1530,1530,877,1372,1530,0


In [53]:
print (len(df_train))

1242


In [34]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [35]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label="label")

label = "mining"

classes = df_test[label].unique().tolist()
print(f"Label classes: {classes}")

df_test[label] = df_test[label].map(classes.index)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_test, label="mining")

Label classes: [False, True]


In [36]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 16 thread(s) for training
Use /tmp/tmpkw1ahqkj as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'nvidia_smi_log.timestamp': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'nvidia_smi_log.gpu.pci.tx_util': <tf.Tensor 'data_1:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.pci.rx_util': <tf.Tensor 'data_2:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.fb_memory_usage.used': <tf.Tensor 'data_3:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.fb_memory_usage.free': <tf.Tensor 'data_4:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.bar1_memory_usage.total': <tf.Tensor 'data_5:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.bar1_memory_usage.used': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.bar1_memory_usage.free': <tf.Tensor 'data_7:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.utilization.gpu_util': <tf.Tensor 'data_8:0' shape=(None,) dtype=int64>, 'nvidia_smi_log.gpu.utilization.memory_uti

[INFO 24-02-13 18:32:02.2663 PST kernel.cc:771] Start Yggdrasil model training
[INFO 24-02-13 18:32:02.2664 PST kernel.cc:772] Collect training examples
[INFO 24-02-13 18:32:02.2664 PST kernel.cc:785] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 24-02-13 18:32:02.2665 PST kernel.cc:391] Number of batches: 2
[INFO 24-02-13 18:32:02.2666 PST kernel.cc:392] Number of examples: 1242
[INFO 24-02-13 18:32:02.2675 PST data_spec_inference.cc:305] 1242 item(s) have been pruned (i.e. they are considered out of dictionary) for the column nvidia_smi_log.timestamp (0 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 24-02-13 18:32:02.2680 

Model trained in 0:00:00.141054
Compiling model...
Model compiled.


In [38]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

2/2 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.9175

loss: 0.0000
accuracy: 0.9175


In [39]:
model_1.make_inspector().variable_importances()

{'INV_MEAN_MIN_DEPTH': [("nvidia_smi_log.gpu.fb_memory_usage.free" (1; #12),
   0.574639254245946),
  ("nvidia_smi_log.gpu.fb_memory_usage.used" (1; #13), 0.5622540138689329),
  ("nvidia_smi_log.gpu.bar1_memory_usage.free" (1; #3), 0.5498869676788667),
  ("nvidia_smi_log.gpu.bar1_memory_usage.used" (1; #5), 0.5485463521667587),
  ("nvidia_smi_log.gpu.clocks.graphics_clock" (1; #6), 0.4982836895138975),
  ("nvidia_smi_log.gpu.clocks.sm_clock" (1; #8), 0.49778761061947),
  ("nvidia_smi_log.gpu.clocks.video_clock" (1; #9), 0.49417966176147676),
  ("nvidia_smi_log.gpu.utilization.memory_util" (1; #29), 0.49328583173472274),
  ("nvidia_smi_log.gpu.pci.tx_util" (1; #20), 0.492287495897605),
  ("nvidia_smi_log.gpu.pci.rx_util" (1; #19), 0.49126637554585234)],
 'NUM_AS_ROOT': [("nvidia_smi_log.gpu.fb_memory_usage.free" (1; #12), 86.0),
  ("nvidia_smi_log.gpu.fb_memory_usage.used" (1; #13), 76.0),
  ("nvidia_smi_log.gpu.bar1_memory_usage.free" (1; #3), 65.0),
  ("nvidia_smi_log.gpu.bar1_memory_

In [40]:
# using a subset of important features
feature_1 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.fb_memory_usage.free")
feature_2 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.fb_memory_usage.used")
feature_3 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.bar1_memory_usage.free")
feature_4 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.bar1_memory_usage.used")
feature_5 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.clocks.sm_clock")
feature_6 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.pci.tx_util")
feature_7 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.clocks.graphics_clock")
feature_8 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.utilization.memory_util")
feature_9 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.clocks.video_clock")
feature_10 = tfdf.keras.FeatureUsage(name="nvidia_smi_log.gpu.temperature.memory_temp")


all_features = [feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8]
model_2 = tfdf.keras.GradientBoostedTreesModel(
    features=all_features, exclude_non_specified_features=True)

model_2.compile(metrics=["accuracy"])
model_2.fit(train_ds, validation_data=test_ds)

print(model_2.evaluate(test_ds, return_dict=True))

Use /tmp/tmpj3j165tc as temporary training directory


[WARNING 24-02-13 18:33:21.5291 PST gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-02-13 18:33:21.5291 PST gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-02-13 18:33:21.5291 PST gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Reading training dataset...


Training dataset read in 0:00:00.217494. Found 1242 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(1164, shape=(), dtype=int32)
Validation dataset read in 0:00:00.411524. Found 1164 examples.
Training model...
Model trained in 0:00:00.201853
Compiling model...


[INFO 24-02-13 18:33:22.8326 PST kernel.cc:1233] Loading model from path /tmp/tmpj3j165tc/model/ with prefix 5aedc1285bf842c5
[INFO 24-02-13 18:33:22.8334 PST quick_scorer_extended.cc:903] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 24-02-13 18:33:22.8335 PST abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-02-13 18:33:22.8335 PST kernel.cc:1061] Use fast generic engine


Model compiled.


2/2 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 0.9175
{'loss': 0.0, 'accuracy': 0.9175257682800293}


In [44]:
# alexa dataset 

alexa = pd.read_csv("alexa-test.csv")

In [45]:
# Dataset to simulate various types of DNS logs

# Dataset of esisting DNS logs
existing = pd.read_csv("dns-dataset - existing.csv")
# Dataset of bad DNS logs
new_bad = pd.read_csv("dns-dataset - new-domains-bad.csv")
# Dataset of new domains good
new_good = pd.read_csv("dns-dataset - new-domains-good.csv")

same = pd.read_csv("dns-dataset - same-domains.csv")

In [47]:
# Check if domain is already existing in list or new domain
def checkDomains(existing, new):
    for index in existing.index:
        value = existing[existing.columns[0]].iloc[index]
        if (new.empty == True):
            pass
        elif (value in new['Domains '].values):
            new = new[new['Domains '] != value]
    new = new.reset_index()
    return (new)

In [83]:
# Check if domain exists in known alexa domain list
def checkAlexa(new):
    if (new.empty):
        return True 
    else:
        for index in new.index: 
            if (new[new.columns[1]].iloc[index] not in alexa['google.com'].values):
                return False
                
    return True

In [88]:
# Check if domain exist in known list or in Alexa list
df = checkDomains(existing, same)
if df.empty:
    print ("Same domain: Domain already exists in known list")
else:
    checkAlexa(df)
 

df = checkDomains(existing, new_good)
if df.empty:
    print ("Domain already exists in known list")
else:
    if (checkAlexa(df)  is True):
        print ("Known domain: Domain exists in Alexa, Not in known list")
    else:
        print ("Malicious domain: Domain does not exist in Alexa or Known list")


df = checkDomains(existing, new_bad)
if (df.empty):
    print ("Domain already exists in known list")
else:
    if (checkAlexa(df)  is True):
        print ("Known domain - Domain exists in Alexa")
    else:
        print ("Malicious domain: Domain does not exist in Alexa or Known list")

Same domain: Domain already exists in known list
Known domain: Domain exists in Alexa, Not in known list
Malicious domain: Domain does not exist in Alexa or Known list
